In [1]:
from olfactorybulb.database import *
import os,sys
from neuronunit.tests.publications import *
from neuronunit.tests.tests import *
from neuronunit.models.neuron_cell import NeuronCellModel
from sciunit.suites import TestSuite
from pandas import DataFrame
import quantities as pq
from neuronunit.tests.utilities import cache
from linetimer import CodeTimer
import string, math
from scipy import optimize
import numpy as np
import matplotlib.pyplot as plt
#import smt
#from smt.surrogate_models.genn import GENN, load_smt_data
from scipy import optimize
import linetimer
import multiprocessing
from multiprocessing import Pool, TimeoutError
from sciunit.scores import ZScore

/home/justas/anaconda2/envs/p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Could not load NEURONBackend


In [2]:
cell_type = 'gc'

# Load tests
measurements = Measurement\
    .select(Measurement)\
    .join(Property)\
    .switch(Measurement)\
    .join(Source)\
    .where((Measurement.property.type == "Electrophysiology") & (Measurement.property.id.startswith(cell_type+'_')))\
    .order_by(Measurement.property.id)

properties = {}

for m in measurements:
    test_generic = str(m.property.test_class_generic)
    pub = str(m.source.publication_class).strip()
    class_name = test_generic+pub
    
    if test_generic not in properties:
        properties[test_generic] = []
    
    globals()[class_name] = type(class_name, 
                                 (eval(pub), eval(test_generic)), 
                                 {})
    
    print('creating specific test class: ' + class_name)
    
    test_instance = eval(class_name)(observation={"mean": m.mean * eval(m.property.units), "std": m.std * eval(m.property.units), "n": m.n})
    
    properties[test_generic].append(test_instance)
    
    
# Load model classes
model_classes = list(CellModel\
                    .select(CellModel)\
                    .where(CellModel.cell_type == cell_type.upper())             
                     )
for i, m in enumerate(model_classes):
    nmsp = string.join(m.isolated_model_class.split('.')[:-1],'.')
    cls = m.isolated_model_class.split('.')[-1]

    import_cmd = 'from '+nmsp+' import '+cls+' as Model'+str(i)
    print(import_cmd)
    exec(import_cmd)
    
# Create work item list
work_items = []

for model in model_classes:
    work_items.append({"model_class": model.isolated_model_class })
        
def get_suite_score(item):
    results = item
    results["properties"] = {}
    results["model_score"] = 0
    
    import prev_ob_models
    exec('cell = '+ str(item["model_class"]) +'()')
    
    if ind is not None:
        from neuron import h
        ind = item["param_values"]

        for pi, pv in enumerate(ind):
            if params[pi]["attr"] == "tau_CaPool":
                setattr(h, params[pi]["attr"], pv)
            else:
                for param_list in params[pi]["lists"]:
                    for sec in getattr(cell.cell, param_list):
                        if params[pi]["attr"] == "diam":
                            for i3d in range(int(h.n3d(sec=sec))):
                                h.pt3dchange(i3d, h.diam3d(i3d, sec=sec)*pv,sec=sec)
                        else:
                            setattr(sec, params[pi]["attr"], pv)
    else:
        ind = []

    model = NeuronCellModel(cell.soma(0.5),name=cell.__class__.__module__+'.'+cell.__class__.__name__+'|'+str(ind))


    for prop in properties.keys():
        
        if prop not in results["properties"]:
            results["properties"][prop] = { "tests": {}, "total_n": 0, "z_score_combined": None}
        
        prop_tests = properties[prop]
        
        for prop_test in prop_tests:
            
            prop_test_result = {}
            results["properties"][prop]["tests"][prop_test.__class__.__name__] = prop_test_result          
            
            try:
                #print('Starting', item, prop_test)
                prediction = prop_test.generate_prediction(model)
                #print('Finished', item, prop_test, prediction)

            except:
                import traceback
                prediction = traceback.format_exc()
                print(prediction)
                
            prop_test_result["observation"] = prop_test.observation
            prop_test_result["prediction"] = prediction
            
            if type(prediction) != str:
                z_score = (prediction - prop_test.observation["mean"])/prop_test.observation["std"]
                z_score = z_score.simplified
            else:
                z_score = 6.0 # errors are treated as 6 std deviation
                
            z_weighed = z_score * prop_test.observation["n"]

            prop_test_result["z_score"] = z_score
            prop_test_result["z_score_weighed"] = z_weighed
            
            results["properties"][prop]["total_n"] += prop_test.observation["n"]
            
        results["properties"][prop]["z_score_combined"] = sum([i["z_score_weighed"] for i in results["properties"][prop]["tests"].values()])
        results["properties"][prop]["z_score_combined"] /= results["properties"][prop]["total_n"]
        
        results["model_score"] += results["properties"][prop]["z_score_combined"].magnitude**2
        
    import math
    results["model_score"] = math.sqrt(results["model_score"])

    return results

creating specific test class: AfterDepolarizationTimeTestStroh2012
creating specific test class: SpikeAmplitudeTestBurtonUrban2015
creating specific test class: SpikeHalfWidthTestBurtonUrban2015
creating specific test class: SpikeThresholdTestBurtonUrban2015
creating specific test class: CellCapacitanceTestBurtonUrban2015
creating specific test class: FISlopeTestBurtonUrban2015
creating specific test class: InputResistanceTestBurtonUrban2015
creating specific test class: InputResistanceTestStroh2012
creating specific test class: RestingVoltageTestBurtonUrban2015
creating specific test class: RestingVoltageTestStroh2012
creating specific test class: RheobaseTestBurtonUrban2015
creating specific test class: SagVoltageTestHu2016
creating specific test class: MembraneTimeConstantTestBurtonUrban2015
from prev_ob_models.Davison2003.isolated_cells import GC as Model0
from prev_ob_models.KaplanLansner2014.isolated_cells import GC as Model1
from prev_ob_models.LiCleland2013.isolated_cells impor

cache.clear()
from prev_ob_models.Birgiolas2020.isolated_cells import MC
result = dowork({"model_class": "prev_ob_models.Birgiolas2020.isolated_cells.MC"})
result

# Show result in a table
df = []
for model in result:
    row = {"model": model["model_class"]}
    for t, test in enumerate(model["properties"].keys()):
        row[test] = model["properties"][test]["z_score_combined"]
    df.append(row)
        
df = DataFrame(df)
df

In [3]:
get_suite_score(work_items[0])

	1 
	0 
	1001 
	0 
	0 
	0 
	0 
	0 
	0 
	0 
	0 
	0 
	0 
	0 
	0 
	0 
	0 


UnboundLocalError: local variable 'ind' referenced before assignment

In [3]:
#cache.clear()

model = Model23()
model.soma

numodel = NeuronCellModel(model.soma(0.5))
numodel

prop_tests = [
    restingSpikesTest,
    rheobaseSpikesTest,
    hz20Spikes,
    properties["InputResistanceTest"][1],
    properties["MembraneTimeConstantTest"][1],
    properties["RestingVoltageTest"][1],
    properties["ReboundSpikingTest"][0],
    properties["CellCapacitanceTest"][1],
    properties["SagVoltageTest"][1],
    properties["SpikeAccommodationTest"][0],
    properties["SpikeAccommodationTimeConstantTest"][0],
    properties["SpikeThresholdTest"][1],
    properties["AfterHyperpolarizationAmplitudeTest"][1],
    properties["AfterHyperpolarizationTimeTest"][1],
    properties["SpikeAmplitudeTest"][1],
    properties["SpikeHalfWidthTest"][1],
    properties["ISICVTest"][1],
    properties["SpikeAccommodationTest"][0],
    properties["SpikeAccommodationTimeConstantTest"][0],
]

for test in prop_tests:
    print('running', test)
    print(test.judge(numodel))


In [3]:
params = [ 
    { "start": 1.0,  "attr": "diam",        "low": 0.1, "high": 2.0, "lists": ["apical","basal","axonal"] },
    { "start": 34.77,  "attr": "Ra",        "low": 5.0, "high": 100.0, "lists": ["all"] },
    { "start": 2.706,   "attr": "cm",        "low": 0.1, "high": 4.0, "lists": ["all"] },
    { "start": 49.95,  "attr": "ena",       "low": 40.0, "high": 50.0, "lists": ["all"] },
    { "start": -70.03,  "attr": "ek",        "low": -100.0, "high": -70.0, "lists": ["all"] },
    { "start": -64.42,  "attr": "e_pas",     "low": -70.0, "high": -50.0, "lists": ["all"] },
    { "start": 0.0005955, "attr": "g_pas",     "low": 0, "high": 0.00003, "lists": ["all"] },    
    { "start": 0.5955,  "attr": "sh_Na",     "low": 0, "high": 10, "lists": ["all"] },    
    { "start": 10,  "attr": "tau_CaPool",     "low": 1, "high": 500, "lists": ["all"] },    

    { "start":  0.87485,  "attr": "gbar_Na",     "low": 0, "high": 0.05, "lists": ["all"] },
    { "start": 0.0297,  "attr": "gbar_Kd",     "low": 0, "high": 0.04, "lists": ["all"] },
    { "start": 0.000264,  "attr": "gbar_Kslow",  "low": 0, "high": 0.004, "lists": ["all"] },
    { "start": 0.07215,  "attr": "gbar_KA",     "low": 0, "high": 0.005, "lists": ["all"] },
    { "start": 0.001,  "attr": "gbar_KCa",    "low": 0, "high": 0.004, "lists": ["all"] },
    { "start": 0.00081441,  "attr": "gbar_LCa",  "low": 0, "high": 0.001, "lists": ["all"] },
    
    { "start": -30.805,  "attr": "eh",       "low": -40.0, "high": -25.0, "lists": ["apical"] },
    { "start": 0.00335,  "attr": "gbar_Ih",    "low": 0, "high": 0.00003, "lists": ["apical"] },
    { "start": 0.000107,  "attr": "gbar_CaT",    "low": 0, "high": 18e-3, "lists": ["apical"] },
]

In [4]:
def evaluate(ind, raw_scores=False):  
    def do_work():
        from prev_ob_models.Birgiolas2020.isolated_cells import MC
        score = get_suite_score({
            "model_class": "prev_ob_models.Birgiolas2020.isolated_cells.MC",
            "param_values": ind
        })
        
        return score["model_score"],
    
    from multiprocess import Pool, TimeoutError
    pool = Pool(processes = 1)
    
    try:
        result = pool.apply_async(do_work).get(timeout=4*60)
    except TimeoutError:
        print('Simulation timed out')
        pool.terminate()
        result = 9*10.0,
    except:
        print('Error in simulation')
        import traceback
        print(traceback.format_exc())
        pool.terminate()
        result = 9*10.999,
        
    return result

In [5]:
#eps = np.sqrt(np.finfo(float).eps)

def evaluate_for_opt(individual):
    return evaluate(individual)[0]
    
def grad_evaluate(individual):
    result = optimize.approx_fprime(individual, evaluate_for_opt, np.abs(np.array(individual)*0.0001))
    
    if np.any(np.abs(result)>1e5):
        print('Discarding invalid gradient', result)
        return [None]*len(individual)
    
    return result

In [5]:
class NoDaemonProcess(multiprocessing.Process):
    # make 'daemon' attribute always return False
    def _get_daemon(self):
        return False
    def _set_daemon(self, value):
        pass
    daemon = property(_get_daemon, _set_daemon)

# We sub-class multiprocessing.pool.Pool instead of multiprocessing.Pool
# because the latter is only a wrapper function, not a proper class.
class MyPool(multiprocessing.pool.Pool):
    Process = NoDaemonProcess

In [6]:
def get_true_points(n):
    dims = len(params)
    
    if type(n) == int:
        print('getting random points')
        xy = np.random.rand(n, dims)

        for pi, p in enumerate(params):
            lower_bound = p["low"]
            upper_bound = p["high"]

            xy[:,pi] = xy[:,pi]*(upper_bound-lower_bound)+lower_bound
    
    else:
        print('evaluating passed-in points')
        xy = n
    
    processes = 15   
    pool = MyPool(processes = processes, maxtasksperchild=1)    
    z = np.array(pool.map(evaluate_for_opt, xy))
    #grad_z = np.array(pool.map(grad_evaluate, xy))
    grad_z = xy
    pool.terminate()
    pool.join()
    
#     valid_z = np.where(z < 10)
#     xy = xy[valid_z]
#     z = z[valid_z]
#     grad_z = grad_z[valid_z]
    
#     valid_grad = grad_z[:,0] != np.array(None)
#     xy = xy[valid_grad]
#     z = z[valid_grad]
#     grad_z = grad_z[valid_grad]
    
    
    return xy, z, grad_z

In [ ]:
# Training
xt, yt, dyt_dxt = get_true_points(150*len(params))

#px, py, pg = get_true_points(np.array([[ 7.09304053e-01,  4.98941113e+01, -7.00000000e+01,  1.65979225e-01,
#        3.30619495e-02,  3.83766954e-05, -7.00000000e+01]]))

# xt = np.concatenate((xt,px))
# yt = np.concatenate((yt,py))
# dyt_dxt = np.concatenate((dyt_dxt,pg))

print('best initial fitness',np.min(yt))
DataFrame(xt,yt)

In [ ]:
def plot3d(x,y,z):
    from matplotlib.mlab import griddata

    xi = np.linspace(x.min(), x.max(), 100)
    yi = np.linspace(y.min(), y.max(), 100)
    zi = griddata(x, y, z, xi, yi, interp='linear')

    #Plot the contour mapping and edit the parameter setting according to your data (http://matplotlib.org/api/pyplot_api.html?highlight=contourf#matplotlib.pyplot.contourf)
    CS = plt.contourf(xi, yi, zi)#, 5, levels=[0,50,100,1000],colors=['b','y','r'],vmax=abs(zi).max(), vmin=-abs(zi).max())
    plt.colorbar()
    plt.show()

In [ ]:
def retrain_surrogate():
    try:
        plot3d(xt[:,0],xt[:,1],yt*-1)
    except:
        pass
    
    # GENN
    genn = GENN()
    genn.options["alpha"] = 0.1             # learning rate that controls optimizer step size
    genn.options["beta1"] = 0.9             # tuning parameter to control ADAM optimization
    genn.options["beta2"] = 0.99            # tuning parameter to control ADAM optimization
    genn.options["lambd"] = 0.1             # lambd = 0. = no regularization, lambd > 0 = regularization
    genn.options["gamma"] = 0.0             # gamma = 0. = no grad-enhancement, gamma > 0 = grad-enhancement
    genn.options["deep"] = 2                # number of hidden layers
    genn.options["wide"] = 10                # number of nodes per hidden layer
    genn.options["mini_batch_size"] = 2000    # used to divide data into training batches (use for large data sets)
    genn.options["num_epochs"] = 10         # number of passes through data
    genn.options["num_iterations"] = 50    # number of optimizer iterations per mini-batch
    genn.options["is_print"] = True         # print output (or not)

    load_smt_data(genn, xt, yt, dyt_dxt)    # convenience function to read in data that is in SMT format
    genn.train()                            # API function to train model
    genn.plot_training_history()            # non-API function to plot training history (to check convergence)
    y_pred = genn.predict_values(xt)         # API function to predict values at new (unseen) points
    
    try:
        plot3d(xt[:,0],xt[:,1],yt*-1)
        plot3d(xt[:,0],xt[:,1],y_pred.flatten()*-1)
    except:
        pass
    
    return genn, y_pred

# genn, y_pred = retrain_surrogate()

# plt.plot(yt, y_pred,'o')
# plt.plot([0,100],[0,100],'-')
# plt.xlim((np.min(yt), np.max(yt)))
# plt.ylim((np.min(y_pred), np.max(y_pred)))
# plt.show()

In [6]:
def GA(suggested_pop=None, n=30, NGEN=30):
    #genn.options['print_global'] = False

    from deap import base, creator
    import math

    creator.create("FitnessMin", base.Fitness, weights=(-1,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    import random
    from deap import tools

    def random_indiv():
        result = [random.random()] * len(params)
        for i, pv in enumerate(result):
            result[i] = (params[i]["high"]-params[i]["low"])*pv+params[i]["low"]
        
        return creator.Individual(result)

    toolbox = base.Toolbox()
    toolbox.register("individual", random_indiv)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    def evaluate_surrogate(individual):
        return genn.predict_values(np.array([individual])).flatten()[0],

    lows = [p["low"] for p in params]
    highs = [p["high"] for p in params]
    
    toolbox.register("mate", tools.cxSimulatedBinaryBounded, eta=0.1, low=lows, up=highs)
    toolbox.register("mutate", tools.mutPolynomialBounded, eta=0.1, low=lows, up=highs, indpb=0.1)
    toolbox.register("evaluate", evaluate)



    toolbox.register("select", tools.selNSGA2, k=int(n*0.2))
    if suggested_pop is None:
        pop = toolbox.population(n=n)
    else:
        pop = [creator.Individual(i) for i in suggested_pop]
        
    CXPB, MUTPB = 1, 1
    F_DIVERSITY = 0.5

    # Evaluate the entire population
    processes = 15   
    pool = MyPool(processes = processes, maxtasksperchild=1)    
    fitnesses = pool.map(toolbox.evaluate, pop)
    pool.terminate()
    pool.join()
    
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(NGEN):
        # Select the parents
        elite = toolbox.select(pop)      

        random_offspring = toolbox.population(n=int(n*F_DIVERSITY/2.0))
        diversity_offspring = random_offspring + tools.selRandom(pop, int(n*F_DIVERSITY/2.0))        
        elite_offspring = tools.selRandom(elite, n-len(elite)-len(diversity_offspring))

        offspring = random_offspring + diversity_offspring + elite_offspring

        # Clone the selected individuals
        offspring = map(toolbox.clone, offspring)

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        
        processes = 15   
        pool = MyPool(processes = processes, maxtasksperchild=1)    
        fitnesses = pool.map(toolbox.evaluate, invalid_ind)
        pool.terminate()
        pool.join()        

        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the parents + offspring
        pop[:] = elite + offspring
        
        print("Generation",g,"out of",NGEN,"COMPLETE")
        print(np.min([i.fitness.values[0] for i in pop]))
        
    

    return pop, pop[0], toolbox.evaluate(pop[0])

In [7]:
top = [[1.0, 34.40577448714667,
  1.7746337170486868,
  48.600061083126825,
  -73.37824874831657,
  -52.149512474246556,
  2.8689454891603497e-05,
  7.600064936952291,
  6.513661312706145,
  0.01835806177392501,
  0.012685911771409467,
  0.001009786028927451,
  0.003485358049971202,
  0.0009111182551995163,
  0.0006405838020076363,
  -36.191181538089175,
  2.6308088272967942e-06,
  0.009520566435955282],
 [1.0, 54.371057327671025,
  1.343586187918468,
  44.40845548134062,
  -81.17397852577881,
  -52.149512474246556,
  2.8689454891603497e-05,
  7.600064936952291,
  12.492102024675049,
  0.01835806177392501,
  0.012243090987314832,
  0.001009786028927451,
  0.0025920677224618473,
  0.0006458839218994715,
  0.0006437525063628547,
  -36.474771746065244,
  1.480641964136122e-06,
  0.009384267411892713],
 [1.0, 
  54.371057327671025,
  1.343586187918468,
  44.40845548134062,
  -81.17397852577881,
  -52.149512474246556,
  2.8689454891603497e-05,
  7.600064936952291,
  12.492102024675049,
  0.01835806177392501,
  0.012243090987314832,
  0.001009786028927451,
  0.0025920677224618473,
  0.0006458839218994715,
  0.0006437525063628547,
  -36.474771746065244,
  1.480641964136122e-06,
  0.009384267411892713]]

In [8]:
pop, pop0, sur_fit = GA(top, n=5, NGEN=1)

Traceback (most recent call last):
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/__init__.py", line 28, in generate_prediction
    result = self.generate_prediction_nocache(model)
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/tests.py", line 372, in generate_prediction_nocache
    return self.get_first_ap(model)
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/tests.py", line 367, in get_first_ap
    raise Exception("The voltage trace does not contain any detectable action potentials using method: " + self.threshold_method)
Exception: The voltage trace does not contain any detectable action potentials using method: dv/dt=20

Traceback (most recent call last):
  File "/home/justas/Repositories/neuronunit_justasb/neuronunit/tests/olfactory_bulb/__init__.py", line 28, in generate_prediction
    result = self.generate_prediction_nocache(model)
  File "/home/justas/Repositories/

In [9]:
import deap
top = deap.tools.selNSGA2(pop,8)
DataFrame([list(i) for i in top],[i.fitness for i in top])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
"(5.491364590786435,)",1.000000,34.405774,1.774634,48.600061,-73.378249,-52.149512,0.000029,7.600065,6.513661,0.018358,0.012686,0.001010,0.003485,0.000911,0.000641,-36.191182,0.000003,0.009521
"(14.319926725409168,)",2.495796,34.405774,1.774634,48.600061,-73.378249,-50.426266,0.000029,7.600065,6.513661,0.018358,0.012686,0.001010,0.003485,0.000911,0.000641,-36.191182,0.000003,0.009521
"(22.159566440580235,)",3.103343,33.819960,1.283135,43.033680,-90.898960,-63.932640,0.000009,3.033680,152.380634,0.015168,0.012135,0.001213,0.001517,0.001213,0.000303,-35.449480,0.000009,0.005461
"(98.991,)",1.000000,54.371057,2.133760,40.701390,-80.008682,-52.149512,0.000029,7.600065,12.492102,0.018358,0.012243,0.001010,0.002592,0.000907,0.000644,-36.474772,0.000003,0.005234
"(98.991,)",1.000000,34.405774,2.877652,48.600061,-75.029214,-52.149512,0.000029,7.600065,6.513661,0.018358,0.012686,0.001010,0.003485,0.000657,0.000640,-36.191182,0.000001,0.009454
"(5011.1892723732135,)",3.103343,33.819960,1.283135,43.033680,-90.898960,-63.932640,0.000009,3.033680,152.380634,0.038205,0.012135,0.001213,0.001517,0.001213,0.000303,-35.449480,0.000009,0.005461


In [18]:
top

[[34.40577448714667,
  1.7746337170486868,
  48.600061083126825,
  -73.37824874831657,
  -52.149512474246556,
  2.8689454891603497e-05,
  7.600064936952291,
  6.513661312706145,
  0.01835806177392501,
  0.012685911771409467,
  0.001009786028927451,
  0.003485358049971202,
  0.0009111182551995163,
  0.0006405838020076363,
  -36.191181538089175,
  2.6308088272967942e-06,
  0.009520566435955282],
 [54.371057327671025,
  1.343586187918468,
  44.40845548134062,
  -81.17397852577881,
  -52.149512474246556,
  2.8689454891603497e-05,
  7.600064936952291,
  12.492102024675049,
  0.01835806177392501,
  0.012243090987314832,
  0.001009786028927451,
  0.0025920677224618473,
  0.0006458839218994715,
  0.0006437525063628547,
  -36.474771746065244,
  1.480641964136122e-06,
  0.009384267411892713],
 [34.40577448714667,
  1.7746337170486868,
  48.25121832219984,
  -85.83988610793635,
  -52.149512474246556,
  2.9121194317970837e-05,
  7.600064936952291,
  11.469065054901318,
  0.01835806177392501,
  0.0

In [ ]:
for i in range(10):
    genn, y_pred = retrain_surrogate()
    
    pop, pop0, sur_fit = GA(xt)

    # Select indivs 
    ga_pop = np.unique(pop,axis=0)

    new_xt, new_yt, new_grad_yt = get_true_points(ga_pop)
    xt =      np.append(xt,     new_xt,     axis=0)
    yt =      np.append(yt,     new_yt,     axis=0)
    dyt_dxt = np.append(dyt_dxt,new_grad_yt,axis=0)
    new_xt, new_yt, new_grad_yt
    
    best = np.argmin(yt)
    
    print("best fitness so far", yt[best])
    
    for pi, p in enumerate(params):
        print(p["attr"], xt[best][pi])

In [ ]:
xt[best]

In [ ]:
DataFrame(xt,yt).sort_index()

In [ ]:
xt.shape

In [ ]:
for c in range(xt.shape[1]):
    plt.plot(xt[:,c],yt,'o',label=params[c]["attr"])
    plt.show()